In [12]:
from playwright.async_api import async_playwright
from io import StringIO
import pandas as pd

pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_colwidth", None) 

# 1. Scrape HTML Crime Data

**Initialise browser and URL**

In [13]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)

In [14]:
page = await browser.new_page()

In [15]:
await page.goto("https://www.crimemapping.com/map/location/2000%20Mission%20St%2C%20San%20Francisco%2C%20CA%2C%2094110%2C%20USA?id=")

<Response url='https://www.crimemapping.com/map/location/2000%20Mission%20St%2C%20San%20Francisco%2C%20CA%2C%2094110%2C%20USA?id=' request=<Request url='https://www.crimemapping.com/map/location/2000%20Mission%20St%2C%20San%20Francisco%2C%20CA%2C%2094110%2C%20USA?id=' method='GET'>>

**Set the "Where" parameters**

500 ft radius around 16th St. BART Plaza with timeouts so things can load dynamically

In [16]:
# click the WHERE filter button
await page.click('#filtersWhere')

await page.wait_for_selector('#wherePanel', state='visible')

await page.fill('#locationSearchWherePanel', '2000 Mission St, San Francisco, CA, 94110, USA')

await page.wait_for_timeout(1000)

await page.press('#locationSearchWherePanel', 'Enter')

await page.click('#whereBufferDistances')

# "500 feet" option has its value as 152.4 in the HTML
await page.select_option('#whereBufferDistances', value='152.4')

await page.wait_for_timeout(500)

# click Apply
await page.click('#wherePanel .btnApply')

await page.wait_for_timeout(1000)

**Set the "When" parameters**

Previous week with timeouts so things can load dynamically

In [17]:
# click the WHEN filter button
await page.click('#filtersWhen')

await page.wait_for_selector('#whenPanel', state='visible')

await page.wait_for_timeout(200)

await page.click('text=Previous 4 Weeks')

await page.wait_for_timeout(200)

**Get the report of crimes**

In [18]:
# click the REPORT button on the left
await page.click('#displayReports')

await page.wait_for_selector('#divReportPage', state='visible')

# wait a bit longer for the data table to fully load. the table sometimes takes a while.
await page.wait_for_timeout(3000)

**Pagination through the table and get the HTML**

In [19]:
records_text = await page.inner_text('.itemsCount')
total_items = int(records_text.split()[0])
items_per_page = 15
total_pages = (total_items + items_per_page - 1) // items_per_page
all_pages = []

for page_num in range(1, total_pages + 1):
    await page.click(f'a[data-page="{page_num}"]')
    await page.wait_for_timeout(2000)
    
    #get the html --> put it in a list
    html = await page.content()
    
    all_pages.append(html)

**Close the browser**

In [95]:
await browser.close()
await playwright.stop()

## 2. Extract and format data from HTML to CSV

In [89]:
all_crimes = pd.read_csv("bart_crime_updated.csv")

In [91]:
# we will be creating a list of dfs and then concat at the end - it's faster

# this will save the most recent crimes that we are scraping - previous 4 weeks only
updated_crimes = [] 

# loop through the html of every page
for page in all_pages:
    
    # pandas will read in the html table cause there is only one per page
    table = pd.read_html(StringIO(page))[1]
    table.drop([0,1], axis=1, inplace=True) # we always want to drop the first two
    
    #rename the columns
    table = table.rename(columns={
    2: 'description',
    3: 'incident_num',
    4: 'location',
    5: 'agency',
    6: 'date'
    })
    
    table[['date', 'time']] = table['date'].str.split(' ', n=1, expand=True)
    
    # now we have the cleaned table of crimes from one page -- add to the list
    updated_crimes.append(table)    

**Add the new crimes to our table of old crimes**

It drops any duplicate incident numbers to make sure that only the new week's crimes are saved.

In [92]:
updated_crimes_df = pd.concat(updated_crimes, ignore_index=True)

# add these new crimes to our main df/csv with all the crimes
all_crimes = pd.concat([all_crimes, updated_crimes_df], ignore_index=True).drop_duplicates(subset='incident_num', keep='last')

**Resave to the original csv.** 

Now that csv is updated with this week's crimes. It will only grow.

In [94]:
all_crimes.to_csv("bart_crime_updated.csv", index = False)